In [ ]:
import torch
print(torch.cuda.is_available())  # Should print 'True' if GPU is available
print(torch.cuda.get_device_name(0))  # Prints the name of the GPU

True
Tesla T4


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("neelpratiksha/indian-traffic-sign-dataset")

print("Path to dataset files:", path)


100%|██████████| 25.8M/25.8M [00:00<00:00, 69.3MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/neelpratiksha/indian-traffic-sign-dataset/versions/1


In [ ]:
import os

# Path where the dataset is stored
dataset_path = '/root/.cache/kagglehub/datasets/neelpratiksha/indian-traffic-sign-dataset/versions/1'

# List the files in the dataset directory
os.listdir(dataset_path)

import shutil

# Define destination path in /content/
destination_path = '/content/indian-traffic-sign-dataset/'

# Move the files from the cache to /content/
shutil.copytree(dataset_path, destination_path)

# List files in the new location
os.listdir(destination_path)


['Indian-Traffic Sign-Dataset']

In [ ]:
import os
import pandas as pd
import snntorch as snn
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from sklearn.model_selection import train_test_split
from PIL import Image
import numpy as np

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float

# Define transformations for the dataset
transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize to match model input size
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1), # Data augmentation
    transforms.RandomRotation(10),  # Data augmentation
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize
])

# Load the CSV for label mappings
csv_file = '/content/indian-traffic-sign-dataset/Indian-Traffic Sign-Dataset/traffic_sign.csv'
labels_df = pd.read_csv(csv_file)

# Create a mapping of ClassId to label name (ClassId is the label)
class_map = dict(zip(labels_df['ClassId'], labels_df['Name']))

# Custom Dataset Class to load images and labels
class TrafficSignDataset(Dataset):
    def __init__(self, images_dir, csv_file, transform=None):
        self.images_dir = images_dir
        self.csv_file = csv_file
        self.transform = transform
        self.images = []
        self.labels = []

        # Load CSV to create a list of image paths and corresponding labels
        self.class_map = pd.read_csv(csv_file)
        self.class_map = dict(zip(self.class_map['ClassId'], self.class_map['Name']))

        # Iterate through each class folder and collect image paths and labels
        for class_id in os.listdir(images_dir):
            class_folder = os.path.join(images_dir, class_id)
            if os.path.isdir(class_folder):
                for img_name in os.listdir(class_folder):
                    img_path = os.path.join(class_folder, img_name)
                    self.images.append(img_path)
                    self.labels.append(int(class_id))  # ClassId is the label

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label

# Load the dataset
image_dir = '/content/indian-traffic-sign-dataset/Indian-Traffic Sign-Dataset/Images'

# Create dataset object
dataset = TrafficSignDataset(images_dir=image_dir, csv_file=csv_file, transform=transform)

# Split into train and test sets (80-20 split)
train_data, test_data = train_test_split(list(zip(dataset.images, dataset.labels)), test_size=0.2, random_state=42)

# Convert back to dataset format
train_images, train_labels = zip(*train_data)
test_images, test_labels = zip(*test_data)

# Create DataLoader for train and test directly using the dataset object
train_loader = DataLoader(dataset, batch_size=64, shuffle=True, drop_last=True)
test_loader = DataLoader(dataset, batch_size=64, shuffle=True, drop_last=True)

# Network parameters
num_inputs = 32 * 32 * 3  # Image size (32x32x3)
num_hidden1 = 1200
num_hidden2 = 800
num_outputs = len(class_map)  # Number of classes (58)

# Temporal Dynamics
num_steps = 50  # Number of time steps in the SNN
beta = 0.9

# Define the Network
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(num_inputs, num_hidden1)
        self.lif1 = snn.Leaky(beta=beta)
        self.fc2 = nn.Linear(num_hidden1, num_hidden2)
        self.lif2 = snn.Leaky(beta=beta)
        self.fc3 = nn.Linear(num_hidden2, num_outputs)
        self.lif3 = snn.Leaky(beta=beta)

    def forward(self, x):
        # Initialize hidden states at t=0
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()
        mem3 = self.lif3.init_leaky()

        # Record the final layer
        spk3_rec = []
        mem3_rec = []

        # Time-loop
        for step in range(num_steps):
            cur1 = self.fc1(x.flatten(1))  # Flatten to batch x (32*32*3)
            spk1, mem1 = self.lif1(cur1, mem1)
            cur2 = self.fc2(spk1)
            spk2, mem2 = self.lif2(cur2, mem2)
            cur3 = self.fc3(spk2)
            spk3, mem3 = self.lif3(cur3, mem3)

            # Store results
            spk3_rec.append(spk3)
            mem3_rec.append(mem3)

        return torch.stack(spk3_rec, dim=0), torch.stack(mem3_rec, dim=0)  # time-steps x batch x num_outputs

# Initialize network, loss function, and optimizer
net = Net().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-4)  # Decreased learning rate

# Training parameters
num_epochs = 10  # Increased epochs
counter = 0

# Outer training loop
for epoch in range(num_epochs):
    net.train()
    for data, targets in train_loader:
        data, targets = data.to(device), targets.to(device)  # Move to device

        # Forward pass
        spk_rec, _ = net(data)

        # Compute loss over time
        loss_val = loss_fn(spk_rec.sum(0), targets)  # Sum spikes over time, then apply CrossEntropy

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss_val.backward()
        optimizer.step()

        # Print loss
        if counter % 100 == 0:
            print(f"Epoch: {epoch} \t Iteration: {counter} \t Train Loss: {loss_val.item()}")
        counter += 1

# Testing the model
net.eval()
correct = 0
total = 0

with torch.no_grad():
    for data, targets in test_loader:
        data, targets = data.to(device), targets.to(device)  # Move to device

        # Forward pass through the network
        spk_rec, _ = net(data)

        # Sum the spikes over time and compute predictions
        output_sum = spk_rec.sum(0)  # Sum over time
        _, predicted = torch.max(output_sum, 1)

        # Update correct predictions and total samples
        correct += (predicted == targets).sum().item()
        total += targets.size(0)

# Calculate accuracy
accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")


Epoch: 0 	 Iteration: 0 	 Train Loss: 4.285878658294678
Epoch: 0 	 Iteration: 100 	 Train Loss: 2.7797276973724365
Epoch: 0 	 Iteration: 200 	 Train Loss: 1.9785447120666504
Epoch: 1 	 Iteration: 300 	 Train Loss: 1.7038651704788208
Epoch: 1 	 Iteration: 400 	 Train Loss: 1.913349986076355
Epoch: 2 	 Iteration: 500 	 Train Loss: 1.4394276142120361
Epoch: 2 	 Iteration: 600 	 Train Loss: 1.1953996419906616
Epoch: 3 	 Iteration: 700 	 Train Loss: 0.8609333038330078
Epoch: 3 	 Iteration: 800 	 Train Loss: 1.0258833169937134
Epoch: 4 	 Iteration: 900 	 Train Loss: 0.751451313495636
Epoch: 4 	 Iteration: 1000 	 Train Loss: 0.9047719240188599
Epoch: 5 	 Iteration: 1100 	 Train Loss: 0.8374596834182739
Epoch: 5 	 Iteration: 1200 	 Train Loss: 0.42405325174331665
Epoch: 5 	 Iteration: 1300 	 Train Loss: 0.6453335285186768
Epoch: 6 	 Iteration: 1400 	 Train Loss: 0.5211565494537354
Epoch: 6 	 Iteration: 1500 	 Train Loss: 0.8875523209571838
Epoch: 7 	 Iteration: 1600 	 Train Loss: 0.38528037071

In [ ]:
import os
import pandas as pd
import snntorch as snn
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from sklearn.model_selection import train_test_split
from PIL import Image
import numpy as np

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float

# Define transformations for the Indian Traffic Sign Dataset
transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize to match model input size
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Data augmentation
    transforms.RandomRotation(10),  # Data augmentation
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize
])

# Load the CSV for label mappings
csv_file = '/content/indian-traffic-sign-dataset/Indian-Traffic Sign-Dataset/traffic_sign.csv'
labels_df = pd.read_csv(csv_file)

# Create a mapping of ClassId to label name (ClassId is the label)
class_map = dict(zip(labels_df['ClassId'], labels_df['Name']))

# Custom Dataset Class to load images and labels
class TrafficSignDataset(Dataset):
    def __init__(self, images_dir, csv_file, transform=None):
        self.images_dir = images_dir
        self.csv_file = csv_file
        self.transform = transform
        self.images = []
        self.labels = []

        # Load CSV to create a list of image paths and corresponding labels
        self.class_map = pd.read_csv(csv_file)
        self.class_map = dict(zip(self.class_map['ClassId'], self.class_map['Name']))

        # Iterate through each class folder and collect image paths and labels
        for class_id in os.listdir(images_dir):
            class_folder = os.path.join(images_dir, class_id)
            if os.path.isdir(class_folder):
                for img_name in os.listdir(class_folder):
                    img_path = os.path.join(class_folder, img_name)
                    self.images.append(img_path)
                    self.labels.append(int(class_id))  # ClassId is the label

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label

# Load the dataset
image_dir = '/content/indian-traffic-sign-dataset/Indian-Traffic Sign-Dataset/Images'

# Create dataset object
dataset = TrafficSignDataset(images_dir=image_dir, csv_file=csv_file, transform=transform)

# Split into train and test sets (80-20 split)
train_data, test_data = train_test_split(list(zip(dataset.images, dataset.labels)), test_size=0.2, random_state=42)

# Convert back to dataset format
train_images, train_labels = zip(*train_data)
test_images, test_labels = zip(*test_data)

# Create DataLoader for train and test directly using the dataset object
train_loader = DataLoader(dataset, batch_size=64, shuffle=True, drop_last=True)
test_loader = DataLoader(dataset, batch_size=64, shuffle=True, drop_last=True)

# Network parameters
num_inputs = 32 * 32 * 3  # Image size (32x32x3)
num_hidden1 = 1200
num_hidden2 = 800
num_outputs = len(class_map)  # Number of classes

# Temporal Dynamics
num_steps = 50  # Number of time steps in the SNN
beta = 0.9

# Define the Network (CNN + SNN)
class ConvNetSNN(nn.Module):
    def __init__(self):
        super().__init__()

        # Convolutional and Pooling Layers with Leaky Integrate-and-Fire (LIF) Neurons
        self.conv1 = nn.Conv2d(3, 8, kernel_size=5, padding="same")  # 3 input channels for RGB
        self.lif1 = snn.Leaky(beta=beta)
        self.mp1 = nn.MaxPool2d(2)

        self.conv2 = nn.Conv2d(8, 24, kernel_size=5, padding="same")
        self.lif2 = snn.Leaky(beta=beta)
        self.mp2 = nn.MaxPool2d(2)

        # Fully Connected Layer
        self.fc = nn.Linear(24 * 8 * 8, num_outputs)  # Adjusted for 32x32 input, after pooling
        self.lif3 = snn.Leaky(beta=beta)

    def forward(self, x):
        # Initialize hidden states at t=0 for each LIF layer
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()
        mem3 = self.lif3.init_leaky()

        # Record the final layer
        spk3_rec = []
        mem3_rec = []

        # Time-step loop
        for step in range(num_steps):
            cur1 = self.conv1(x)
            spk1, mem1 = self.lif1(self.mp1(cur1), mem1)
            cur2 = self.conv2(spk1)
            spk2, mem2 = self.lif2(self.mp2(cur2), mem2)
            cur3 = self.fc(spk2.flatten(1))
            spk3, mem3 = self.lif3(cur3, mem3)

            spk3_rec.append(spk3)
            mem3_rec.append(mem3)

        return torch.stack(spk3_rec, dim=0), torch.stack(mem3_rec, dim=0)

# Initialize Network
convnet_snn = ConvNetSNN().to(device)

# Loss and Optimizer
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(convnet_snn.parameters(), lr=1e-3)

# Training Loop
num_epochs = 10
loss_hist = []
counter = 0

for epoch in range(num_epochs):
    for data, targets in train_loader:
        data = data.to(device)
        targets = targets.to(device)

        # Forward pass
        convnet_snn.train()
        spk_rec, _ = convnet_snn(data)

        # Calculate loss and perform optimization
        loss_val = loss(spk_rec.sum(0), targets)  # Sum spikes over time
        optimizer.zero_grad()
        loss_val.backward()
        optimizer.step()

        loss_hist.append(loss_val.item())

        # Print progress
        if counter % 100 == 0:
            print(f"Iteration: {counter}  Train Loss: {loss_val.item()}")
        counter += 1

# Accuracy Measurement Function
def measure_accuracy(model, dataloader):
    with torch.no_grad():
        model.eval()
        correct = 0
        total = 0

        for data, targets in dataloader:
            data = data.to(device)
            targets = targets.to(device)

            # Forward pass
            spk_rec, _ = model(data)
            spike_count = spk_rec.sum(0)
            _, predicted = spike_count.max(1)

            correct += (predicted == targets).sum().item()
            total += targets.size(0)

    return correct / total

# Evaluate Network
accuracy = measure_accuracy(convnet_snn, test_loader)
print(f"ConvNetSNN Accuracy on Indian Traffic Sign Dataset: {accuracy * 100:.2f}%")


Iteration: 0  Train Loss: 5.187477111816406
Iteration: 100  Train Loss: 2.205054759979248
Iteration: 200  Train Loss: 1.4201500415802002
Iteration: 300  Train Loss: 1.2325161695480347
Iteration: 400  Train Loss: 1.19049072265625
Iteration: 500  Train Loss: 1.3851219415664673
Iteration: 600  Train Loss: 0.5304021835327148
Iteration: 700  Train Loss: 0.8227061629295349
Iteration: 800  Train Loss: 0.6850812435150146
Iteration: 900  Train Loss: 0.5536622405052185
Iteration: 1000  Train Loss: 0.8062125444412231
Iteration: 1100  Train Loss: 0.45163410902023315
Iteration: 1200  Train Loss: 0.42234915494918823
Iteration: 1300  Train Loss: 0.44191619753837585
Iteration: 1400  Train Loss: 0.5727237462997437
Iteration: 1500  Train Loss: 0.5735800862312317
Iteration: 1600  Train Loss: 0.5465282201766968
Iteration: 1700  Train Loss: 0.5095245838165283
Iteration: 1800  Train Loss: 0.3520330488681793
Iteration: 1900  Train Loss: 0.5468196868896484
Iteration: 2000  Train Loss: 0.2330668568611145
Itera